In [1]:
import random
import torch


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


In [2]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

In [4]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0109, -0.0139, -0.0191,  ..., -0.0023, -0.0235,  0.0018],
         [-0.0314, -0.0315, -0.0205,  ..., -0.0210,  0.0030, -0.0138],
         [ 0.0055,  0.0101, -0.0076,  ...,  0.0078, -0.0079,  0.0186],
         ...,
         [-0.0183,  0.0287,  0.0206,  ...,  0.0298, -0.0092, -0.0170],
         [-0.0093,  0.0301,  0.0188,  ..., -0.0136,  0.0173,  0.0213],
         [-0.0145,  0.0163, -0.0263,  ...,  0.0289, -0.0011, -0.0294]],
        requires_grad=True), Parameter containing:
 tensor([ 0.0116, -0.0097, -0.0021,  0.0158, -0.0227,  0.0190,  0.0161, -0.0211,
          0.0035, -0.0145, -0.0154,  0.0054,  0.0039,  0.0270,  0.0079,  0.0275,
          0.0243, -0.0119,  0.0113, -0.0286,  0.0129, -0.0051, -0.0073, -0.0006,
          0.0184, -0.0270,  0.0208, -0.0239,  0.0006,  0.0027,  0.0026,  0.0100,
         -0.0179,  0.0054,  0.0190, -0.0205,  0.0083,  0.0163,  0.0149, -0.0139,
          0.0004,  0.0242, -0.0286,  0.0050,  0.0091,  0.0313,  0.0301,  0.0159,